# **Start**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Uninstall existing scikit-learn to avoid conflicts
!pip uninstall -y scikit-learn
# Install specific versions of libraries to avoid conflicts
!pip install scikit-learn==1.5.2
!pip install bayesian-optimization
!pip install optuna
!pip install gpboost==1.6.1
!pip install shap
!pip install ngboost
!pip install dask[dataframe]
!pip install torch seaborn
!pip install lightgbm
!pip install xgboost
!pip install lime
!pip install interpret
!pip install optunahub
!pip install cmaes
!pip install plotly kaleido
!pip install openpyxl
!pip install -U kaleido
!pip install properscoring
!pip install XlsxWriter
!pip install cython
!pip install pgbm
!pip install torch
!pip install cp
!pip install mapie
!pip install torch skorch puncc
# Reinstall scikit-learn to the version required by ngboost
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.6.1
# Reinstall numpy first
!pip install numpy==1.26.4  # Use the version compatible with catboost
# Reinstall catboost
!pip install catboost
!pip install pytorch-tabnet2

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 115.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 12.2 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-non

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 9.9 MB/s eta 0:00:00


In [ ]:
# Restart the runtime to apply changes
import os
os._exit(00)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ngboost
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from gpboost import GPBoostRegressor
from ngboost import NGBRegressor
import optuna
import optunahub
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from interpret import show
from interpret.blackbox import LimeTabular, ShapKernel
from optuna.samplers import RandomSampler
import random
import time
from ngboost.distns import Normal
from ngboost.scores import LogScore
from scipy.stats import norm
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
import plotly.express as px
from io import BytesIO
from openpyxl import Workbook, load_workbook
import os
from openpyxl.drawing.image import Image as openpyxlImage
import warnings
import xlsxwriter
from openpyxl.drawing.image import Image
from pgbm.sklearn import HistGradientBoostingRegressor
import torch
from pgbm.torch import PGBM
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import pickle
import json
from pytorch_tabnet import TabNetRegressor

W1110 08:50:01.720000 3183 torch/utils/cpp_extension.py:118] No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
train_data_path = "./drive/MyDrive/streamflow/TDS/train.csv"
test_data_path = "./drive/MyDrive/streamflow/TDS/test.csv"
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
print("Training data loaded successfully.")
print("Test data loaded successfully.")

Training data loaded successfully.
Test data loaded successfully.


In [3]:
print("\nShape of training data:", train_data.shape)
print("First 5 rows of training data:\n", train_data.head(5))
print("\nShape of test data:", test_data.shape)
print("First 5 rows of test data:\n", test_data.head(5))


Shape of training data: (192, 5)
First 5 rows of training data:
     pH  Salinity  Turbidity   Water Temperature  TDS
0  7.6       0.2         3.0               19.9  290
1  7.4       0.1         0.5               20.7  120
2  7.3       0.2        10.7               23.6  200
3  8.0       0.1         3.0               27.0   94
4  7.8       0.4         1.4               28.4  330

Shape of test data: (96, 5)
First 5 rows of test data:
     pH  Salinity  Turbidity   Water Temperature  TDS
0  7.2       0.1         9.1               18.8  160
1  7.4       0.2         3.7               26.3  190
2  7.0       0.1         3.0               21.6  180
3  8.1       0.1         2.2               17.2  110
4  7.9       0.1        14.1               27.0  200


In [4]:
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
x_test= X_test
print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (192, 4)
Shape of y_train: (192,)
Shape of X_test: (96, 4)
Shape of y_test: (96,)


In [5]:
# Apply z-score normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Print the first five rows of the normalized data
print("\nFirst five rows of normalized X_train:")
print(X_train[:5])

print("\nFirst five rows of normalized X_test:")
print(X_test[:5])


First five rows of normalized X_train:
[[-0.1272241   0.06262254 -0.51513341 -1.14481478]
 [-0.57957647 -0.38434874 -0.6817983  -0.97088446]
 [-0.80575265  0.06262254 -0.00180554 -0.34038706]
 [ 0.77748062 -0.38434874 -0.51513341  0.39881678]
 [ 0.32512826  0.95656508 -0.62179894  0.70319483]]

First five rows of normalized X_test:
[[-1.03192883 -0.38434874 -0.10847107 -1.38396896]
 [-0.57957647  0.06262254 -0.46846724  0.24662775]
 [-1.48428119 -0.38434874 -0.51513341 -0.77521285]
 [ 1.00365681 -0.38434874 -0.56846617 -1.73182959]
 [ 0.55130444 -0.38434874  0.22485872  0.39881678]]


# **Functions**

In [6]:
# Define the model classes
model_classes = {
    'Random Forest': RandomForestRegressor,
    'Gradient Boosting': GradientBoostingRegressor,
    'XGBoost': XGBRegressor,
    'LightGBM': LGBMRegressor,
    'GPBoost': GPBoostRegressor,
    'CatBoost': CatBoostRegressor,
    'NGBoost': NGBRegressor,
    'TabNet' : TabNetRegressor
}

feature_names = ['ph', 'Salinity', 'Turbidity', 'Water Temperature']


In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

def plot_best_scores(best_scores_ran, excel_file_path):
    # Find best scores for each model (by pruner)
    best_rmse_scores = {}
    best_corr_coef_scores = {}

    for (model_name, pruner_name), scores in best_scores_ran.items():
        # Defensive: skip if score info missing
        if "test_rmse" not in scores or "test_corr_coef" not in scores:
            print(f"Skipping {model_name} ({pruner_name}): missing required test metrics.")
            continue

        # Best RMSE (minimize)
        if model_name not in best_rmse_scores or scores['test_rmse'] < best_rmse_scores[model_name][0]:
            best_rmse_scores[model_name] = (scores['test_rmse'], pruner_name)
        # Best Correlation (maximize)
        if model_name not in best_corr_coef_scores or scores['test_corr_coef'] > best_corr_coef_scores[model_name][0]:
            best_corr_coef_scores[model_name] = (scores['test_corr_coef'], pruner_name)

    # -- Defensive checks --
    if not best_rmse_scores or not best_corr_coef_scores:
        print("No models found with valid scores, exiting function.")
        return

    # For RMSE plot
    model_names_rmse = [f"{m} ({p})" for m, (rmse, p) in best_rmse_scores.items()]
    rmse_values = [rmse for rmse, _ in best_rmse_scores.values()]
    # For Corr plot
    model_names_corr = [f"{m} ({p})" for m, (corr, p) in best_corr_coef_scores.items()]
    corr_values = [corr for corr, _ in best_corr_coef_scores.values()]

    generated_images = []

    # ----- RMSE Plot -----
    if model_names_rmse and rmse_values:
        fig, ax = plt.subplots(figsize=(12, 6))
        bars = ax.bar(model_names_rmse, rmse_values, color='skyblue')
        best_idx = int(np.argmin(rmse_values))
        bars[best_idx].set_color('orange')
        for i, bar in enumerate(bars):
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.05,
                    f'{rmse_values[i]:.5f}', ha='center', va='bottom', color='black')
        plt.xlabel('Model (Pruner)')
        plt.ylabel('Test RMSE')
        plt.title('Best Test RMSE for Each Model')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        rmse_image_path = 'rmse_plot.png'
        plt.savefig(rmse_image_path)
        plt.close()
        generated_images.append(('RMSE', rmse_image_path))

    # ----- Corr Plot -----
    if model_names_corr and corr_values:
        fig, ax = plt.subplots(figsize=(12, 6))
        bars = ax.bar(model_names_corr, corr_values, color='lightgreen')
        best_idx = int(np.argmax(corr_values))
        bars[best_idx].set_color('orange')
        for i, bar in enumerate(bars):
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.05,
                    f'{corr_values[i]:.5f}', ha='center', va='bottom', color='black')
        plt.xlabel('Model (Pruner)')
        plt.ylabel('Correlation Coefficient')
        plt.title('Best Correlation Coefficient for Each Model')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        corr_image_path = 'corr_plot.png'
        plt.savefig(corr_image_path)
        plt.close()
        generated_images.append(('CORR', corr_image_path))

    # ---- Insert into Excel (only actually created images) ----
    workbook = load_workbook(excel_file_path)
    sheet_name = 'Best Models Plots'
    sheet = workbook[sheet_name] if sheet_name in workbook.sheetnames else workbook.create_sheet(title=sheet_name)
    start_rows = {'RMSE': 1, 'CORR': 20}

    for desc, img_path in generated_images:
        if os.path.exists(img_path):
            img = Image(img_path)
            pos = f"A{start_rows[desc]}"
            sheet.add_image(img, pos)
        else:
            print(f"Warning: Image file {img_path} not found; skipping insertion.")

    workbook.save(excel_file_path)

    # Clean up any generated images
    for _, img_path in generated_images:
        if os.path.exists(img_path):
            os.remove(img_path)

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

def generate_interpretml_explanations_summary_pruners(
    results_dict, X_train, y_train, X_test, feature_names, instance_indices=None, excel_file_path=None
):
    if instance_indices is None:
        instance_indices = range(len(X_test))
    elif isinstance(instance_indices, int):
        instance_indices = [instance_indices]

    valid_indices = [idx for idx in instance_indices if 0 <= idx < len(X_test)]
    if not valid_indices:
        print("No valid instance indices provided.")
        return

    if isinstance(X_test, pd.DataFrame):
        instances_to_explain = X_test.iloc[valid_indices]
    else:
        instances_to_explain = X_test[valid_indices]

    best_model_pruners = {}
    for model_key, model_info in results_dict.items():
        if isinstance(model_key, tuple):
            model_name, pruner_name = model_key
        else:
            model_name = model_key
            pruner_name = None

        best_score = model_info.get('best_score')
        if best_score is None:
            print(f"No 'best_score' found for {model_key}. Skipping this combination.")
            continue

        if model_name not in best_model_pruners:
            best_model_pruners[model_name] = {
                'pruner_name': pruner_name,
                'model_info': model_info,
                'best_score': best_score
            }
        else:
            current_best_score = best_model_pruners[model_name]['best_score']
            if best_score < current_best_score:
                best_model_pruners[model_name] = {
                    'pruner_name': pruner_name,
                    'model_info': model_info,
                    'best_score': best_score
                }

    for model_name, info in best_model_pruners.items():
        pruner_name = info['pruner_name']
        model_info = info['model_info']
        best_params = dict(model_info['best_params'])  # don't mutate original!
        model_class = model_classes.get(model_name)

        if model_class is None:
            print(f"Model {model_name} is not supported or not available.")
            continue

        if model_name == 'CatBoost':
            best_params['verbose'] = 0

        # ------- Main model fit logic ---------
        if model_name == "TabNet":
            # TabNet: reshape y, fit, flatten pred for LIME/SHAP, etc.
            y_train_tabnet = np.array(y_train).reshape(-1, 1)
            try:
                model = model_class(**{k: v for k, v in best_params.items() if k != "verbose"})
            except TypeError:
                model = model_class()
            model.fit(np.array(X_train), y_train_tabnet, max_epochs=100, patience=10, batch_size=1024, eval_set=[(np.array(X_train), y_train_tabnet)])
            def predict_fn(data):
                preds = model.predict(np.array(data))
                # flatten for interpreters
                return preds.flatten()
        else:
            try:
                model = model_class(**best_params)
            except TypeError:
                model = model_class()
            model.fit(X_train, y_train)
            def predict_fn(data):
                return model.predict(data)

        if isinstance(X_train, pd.DataFrame):
            data_for_explainer = X_train.values
        else:
            data_for_explainer = X_train

        if isinstance(instances_to_explain, pd.DataFrame):
            data_for_explanation = instances_to_explain.values
        else:
            data_for_explanation = instances_to_explain

        # Generate LIME explanations
        lime_explainer = LimeTabular(
            predict_fn,
            data=data_for_explainer,
            feature_names=feature_names,
            random_state=1,
            mode='regression'
        )
        lime_explanation = lime_explainer.explain_local(data_for_explanation)

        feature_importances_lime = {}
        num_instances = len(valid_indices)
        for idx in range(num_instances):
            explanation = lime_explanation.data(idx)
            for feature_name, feature_score in zip(explanation['names'], explanation['scores']):
                feature_importances_lime[feature_name] = feature_importances_lime.get(feature_name, 0) + abs(feature_score)
        feature_importances_lime = {k: v / num_instances for k, v in feature_importances_lime.items()}
        feature_importances_lime = {k: round(v, 3) for k, v in feature_importances_lime.items()}

        # Generate SHAP explanations using ShapKernel
        try:
            shap_explainer = ShapKernel(predict_fn, data_for_explainer, feature_names=feature_names)
            shap_explanation = shap_explainer.explain_local(data_for_explanation)

            feature_importances_shap = {}
            for idx in range(num_instances):
                explanation = shap_explanation.data(idx)
                for feature_name, feature_score in zip(explanation['names'], explanation['scores']):
                    feature_importances_shap[feature_name] = feature_importances_shap.get(feature_name, 0) + abs(feature_score)

            feature_importances_shap = {k: v / num_instances for k, v in feature_importances_shap.items()}
            feature_importances_shap = {k: round(v, 3) for k, v in feature_importances_shap.items()}
        except Exception as e:
            print(f"Could not compute SHAP values for model {model_name}: {e}")
            feature_importances_shap = {}

        # Plot LIME and SHAP feature importances side by side
        fig, axes = plt.subplots(1, 2, figsize=(34, 36))

        # Plot LIME feature importances
        lime_importances_df = pd.DataFrame.from_dict(
            feature_importances_lime, orient='index', columns=['importance']
        )
        lime_importances_df.sort_values(by='importance', ascending=False, inplace=True)
        lime_importances_df.plot(kind='bar', legend=False, color='skyblue', ax=axes[0])
        axes[0].set_title(f"LIME Feature Importances for {model_name}")
        axes[0].set_ylabel("Average Absolute Importance Score")
        axes[0].set_xlabel("Features")
        axes[0].tick_params(axis='x', rotation=45)

        for p in axes[0].patches:
            height = p.get_height()
            axes[0].annotate(f'{height:.3f}',
                             (p.get_x() + p.get_width() / 2., height),
                             ha='center', va='bottom', fontsize=8)

        # Plot SHAP feature importances
        shap_importances_df = pd.DataFrame.from_dict(
            feature_importances_shap, orient='index', columns=['importance']
        )
        shap_importances_df.sort_values(by='importance', ascending=False, inplace=True)
        shap_importances_df.plot(kind='bar', legend=False, color='orange', ax=axes[1])
        axes[1].set_title(f"SHAP Feature Importances for {model_name}")
        axes[1].set_ylabel("Average Absolute SHAP Value")
        axes[1].set_xlabel("Features")
        axes[1].tick_params(axis='x', rotation=45)

        for p in axes[1].patches:
            height = p.get_height()
            axes[1].annotate(f'{height:.3f}',
                             (p.get_x() + p.get_width() / 2., height),
                             ha='center', va='bottom', fontsize=8)

        plt.tight_layout()

        # Save plots as images
        image_path = f'feature_importances_{model_name}.png'
        fig.savefig(image_path)
        plt.close(fig)

        # Optionally insert images and scores into an Excel file
        if excel_file_path:
            workbook = load_workbook(excel_file_path)
            sheet_name = f'{model_name} Explanations'
            if sheet_name in workbook.sheetnames:
                sheet = workbook[sheet_name]
            else:
                sheet = workbook.create_sheet(title=sheet_name)

            # Insert images into the new Excel sheet
            img = Image(image_path)
            sheet.add_image(img, 'A1')

            # Create a new sheet for feature importance scores
            scores_sheet_name = f'{model_name} Scores'
            if scores_sheet_name in workbook.sheetnames:
                scores_sheet = workbook[scores_sheet_name]
            else:
                scores_sheet = workbook.create_sheet(title=scores_sheet_name)

            # Write LIME scores
            scores_sheet.append(['Feature', 'LIME Importance'])
            for feature, importance in feature_importances_lime.items():
                scores_sheet.append([feature, importance])

            # Write SHAP scores if available
            if feature_importances_shap:
                scores_sheet.append(['Feature', 'SHAP Importance'])
                for feature, importance in feature_importances_shap.items():
                    scores_sheet.append([feature, importance])

            # Save the workbook
            workbook.save(excel_file_path)

            # Clean up the image file
            os.remove(image_path)

# **Autosampler by Optuna**

In [9]:
# Suppress Optuna's verbose output
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Define objective and metric functions for PGBM
def mseloss_objective(yhat, y, sample_weight=None):
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    gradient = yhat - y
    hessian = torch.ones_like(yhat)
    return gradient, hessian

def rmseloss_metric(yhat, y, sample_weight=None):
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    loss = torch.sqrt(torch.mean((yhat - y) ** 2))
    return loss

def hyperparameter_tuning_all(X_train, y_train, X_test, y_test, excel_path):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    models = {
        'Random Forest': (RandomForestRegressor, {
            'n_estimators': [100, 200, 300, 500, 700],
            'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
            'max_depth': [None, 10, 20, 30, 40],
            'min_samples_split': [2, 5, 10, 0.01],
            'min_samples_leaf': [1, 3, 5, 0.01],
            'min_weight_fraction_leaf': [0.0, 0.01, 0.1, 0.2],
            'max_features': [1.0, 'sqrt', 'log2', 0.3, 0.5],
            'max_leaf_nodes': [None, 50, 100, 200],
            'min_impurity_decrease': [0.0, 0.01, 0.1, 0.2],
            'n_jobs': [-1],
            'random_state': [42],
            'verbose': [0],
            'warm_start': [False],
            'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1]
        }),
        'Gradient Boosting': (GradientBoostingRegressor, {
            'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'n_estimators': [100, 200, 300, 500, 700],
            'subsample': [1.0, 0.9, 0.7, 0.5],
            'criterion': ['friedman_mse', 'squared_error'],
            'min_samples_split': [2, 5, 10, 0.01],
            'min_samples_leaf': [1, 3, 5, 0.01],
            'min_weight_fraction_leaf': [0.0, 0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7, 10],
            'min_impurity_decrease': [0.0, 0.01, 0.1],
            'init': [None],
            'random_state': [42],
            'max_features': [None, 'sqrt', 'log2', 0.5],
            'alpha': [0.9, 0.5, 0.1],
            'verbose': [0],
            'max_leaf_nodes': [None, 10, 30, 50],
            'warm_start': [False],
            'validation_fraction': [0.1],
            'n_iter_no_change': [None, 10, 20],
            'tol': [1e-4, 1e-3],
            'ccp_alpha': [0.0, 0.001, 0.01]
        }),
        'XGBoost': (XGBRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3, 5],
            'gamma': [0, 0.1, 0.5, 1],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
            'colsample_bytree': [0.5, 0.7, 0.9],
            'colsample_bylevel': [0.5, 0.7, 0.9],
            'reg_alpha': [0, 0.01, 0.1, 1],
            'reg_lambda': [0.1, 1, 5, 10],
            'objective': ['reg:squarederror'],
            'random_state': [42],
            'n_jobs': [-1]
        }),
        'LightGBM': (LGBMRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'num_leaves': [15, 31, 63],
            'max_depth': [3, 5, 7, -1],
            'min_child_samples': [1, 5, 10, 20],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
            'reg_alpha': [0, 0.01, 0.1, 1],
            'reg_lambda': [0, 0.1, 1, 10],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1],
            'bagging_freq': [0, 1, 5],
            'objective': ['regression'],
            'random_state': [42],
            'n_jobs': [-1],
            'verbose': [-1]
        }),
        'GPBoost': (GPBoostRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7, -1],
            'num_leaves': [15, 31, 63],
            'min_child_samples': [1, 5, 10, 20],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
            'reg_alpha': [0, 0.1, 0.5, 1.0],
            'reg_lambda': [0, 0.1, 0.5, 1.0],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1],
            'random_state': [42],
            'n_jobs': [-1],
            'verbose': [-1]
        }),
        'CatBoost': (CatBoostRegressor, {
            'iterations': [200, 500, 1000],
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'depth': [4, 6, 8, 10],
            'l2_leaf_reg': [1, 3, 5, 7, 9],
            'border_count': [32, 64, 128],
            'min_data_in_leaf': [1, 5, 10, 20],
            'rsm': [0.6, 0.8, 1.0],
            'bagging_temperature': [0, 1, 10],
            'random_seed': [42],
            'verbose': [0]
        }),
        'NGBoost': (NGBRegressor, {
            'n_estimators': [200, 500, 1000],
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'natural_gradient': [True, False],
            'minibatch_frac': [0.5, 0.7, 0.9, 1.0],
            'col_sample': [0.5, 0.7, 0.9, 1.0],
            'Dist': [Normal],
            'Score': [LogScore],
            'random_state': [42],
            'verbose': [0]
        }),
        'HistGradientBoosting': (HistGradientBoostingRegressor, {
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_iter': [100, 200, 300, 400, 500],
            'max_depth': [3, 5, 7, None],
            'min_samples_leaf': [5, 10, 20],
            'max_leaf_nodes': [15, 31, 63, None],
            'l2_regularization': [0.0, 0.1, 0.5, 1.0],
            'max_bins': [64, 128, 255],
            'early_stopping': [True, False],
            'validation_fraction': [0.1, 0.2],
            'n_iter_no_change': [5, 10, 15],
            'loss': ['squared_error'],
            'random_state': [42],
            'verbose': [0]
        }),
        'PGBM': (PGBM, {}), # PGBM is handled separately
        "TabNet": (TabNetRegressor, {
            'n_d': [8, 16, 32, 64],
            'n_a': [8, 16, 32, 64],
            'n_steps': [3, 5, 7, 10],
            'gamma': [1.0, 1.3, 1.5, 2.0],
            'lambda_sparse': [1e-4, 1e-3, 1e-2],
            'optimizer_params': [{'lr': 2e-2}], # Fixed learning rate as recommended
            'mask_type': ['sparsemax', 'entmax'],
            'n_shared': [1, 2, 3],
            'n_independent': [1, 2, 3],
            'scheduler_params': [{"step_size": 10, "gamma": 0.9}],
            'scheduler_fn': [torch.optim.lr_scheduler.StepLR],
            'seed': [42],
            'verbose': [0]
        })
    }


    pruners = [
        optuna.pruners.MedianPruner(),
        optuna.pruners.NopPruner(),
        optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=3),
        optuna.pruners.PercentilePruner(25.0),
        optuna.pruners.SuccessiveHalvingPruner(),
        optuna.pruners.HyperbandPruner(),
        optuna.pruners.ThresholdPruner(lower=0.1),
        optuna.pruners.WilcoxonPruner()
    ]

    best_scores = {}
    predictions_df = pd.DataFrame({'Actual': y_test})

    for model_name, (model_class, param_space) in models.items():
        for pruner in pruners:
            pruner_name = pruner.__class__.__name__
            print(f"Running Optuna for {model_name} with {pruner_name}...")
            try:
                sampler = optunahub.load_module("samplers/auto_sampler").AutoSampler()
                study = optuna.create_study(direction='minimize', sampler=sampler, pruner=pruner)

                if model_name == 'PGBM':
                    def pgbm_objective(trial):
                        params = {
                            'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300, 500]),
                            'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1, 0.15]),
                            'max_leaves': trial.suggest_int('max_leaves', 15, 63),
                            'min_split_gain': trial.suggest_categorical('min_split_gain', [0.0, 0.1, 0.5, 1.0]),
                            'reg_lambda': trial.suggest_categorical('reg_lambda', [0.1, 1.0, 5.0, 10.0]),
                            'feature_fraction': trial.suggest_categorical('feature_fraction', [0.5, 0.7, 0.9, 1.0]),
                            'bagging_fraction': trial.suggest_categorical('bagging_fraction', [0.5, 0.7, 0.9, 1.0]),
                            'tree_correlation': trial.suggest_categorical('tree_correlation', [0.0, 0.1, 0.2, 0.3]),
                            'min_data_in_leaf': trial.suggest_categorical('min_data_in_leaf', [3, 5, 10, 20]),
                            'max_bin': trial.suggest_categorical('max_bin', [64, 128, 256]),
                            'distribution': trial.suggest_categorical('distribution', ['normal', 'studentt', 'laplace']),
                            'objective': 'mse',
                            'metric': 'rmse',
                            'random_state': 42,
                            'verbose': 0
                        }

                        model = PGBM()
                        model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=params)
                        y_pred = model.predict(X_test)
                        mse = mean_squared_error(y_test, y_pred)
                        return mse

                    study.optimize(pgbm_objective, n_trials=50)
                else:
                    def objective(trial, model_name, model_class, param_space):
                        params = {}
                        for key, values in param_space.items():
                            if isinstance(values, list):
                                params[key] = trial.suggest_categorical(key, values)
                            # Add other suggestion types if needed for other models...

                        # Special handling for TabNet's data shape requirement
                        if model_name == 'TabNet':
                            # Reshape target variables to 2D for TabNet
                            y_train_fit = y_train.reshape(-1, 1)
                            y_test_fit = y_test.reshape(-1, 1)

                            model = model_class(**params)
                            # Use the reshaped data for fitting and evaluation
                            model.fit(
                                X_train, y_train_fit,
                                eval_set=[(X_test, y_test_fit)],
                                patience=10,
                                max_epochs=100
                            )
                            y_pred = model.predict(X_test)
                            mse = mean_squared_error(y_test_fit, y_pred)
                        else:
                            # Standard fitting for all other models
                            model = model_class(**params)
                            model.fit(X_train, y_train)
                            y_pred = model.predict(X_test)
                            mse = mean_squared_error(y_test, y_pred)

                        return mse

                    study.optimize(lambda trial: objective(trial, model_name, model_class, param_space), n_trials=50)

                best_params = study.best_params
                # Create an instance of the best model
                best_model = model_class(**best_params) if model_name != 'PGBM' else PGBM()

                # Retrain the best model on the full data
                if model_name == 'PGBM':
                    best_model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=best_params)
                elif model_name == 'TabNet':
                    # *** THIS IS THE FIX: Reshape y_train just for TabNet's final fit ***
                    best_model.fit(X_train, y_train.reshape(-1, 1))
                else:
                    # All other models use the original y_train
                    best_model.fit(X_train, y_train)

                y_pred = best_model.predict(X_test)
                # *** THIS IS THE FIX: Flatten the prediction array to 1D ***
                y_pred_flat = y_pred.flatten()

                # Now use the flattened array for all subsequent operations
                mse = mean_squared_error(y_test, y_pred_flat)
                rmse = np.sqrt(mse)
                corr_coef = np.corrcoef(y_test, y_pred_flat)[0, 1]

                # Save the flattened predictions to the DataFrame
                predictions_df[f'{model_name}_{pruner_name}_Predicted'] = y_pred_flat

                best_scores[(model_name, pruner_name)] = {
                    'best_score': mse,
                    'best_params': best_params,
                    'test_mse': mse,
                    'test_rmse': rmse,
                    'test_corr_coef': corr_coef,
                    'pruner': pruner_name
                }
                print(f"Best MSE for {model_name} with {pruner_name}: {mse} with params: {best_params}")
                print(f"Best RMSE for {model_name} with {pruner_name}: {rmse}")
                print(f"Correlation Coefficient for {model_name} with {pruner_name}: {corr_coef}")
            except Exception as e:
                print(f"Failed to run Optuna for {model_name} with {pruner_name}. Error: {e}")

    # Write predictions to Excel
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        predictions_df.to_excel(writer, sheet_name='Predictions', index=False)

    if best_scores:
        best_model_name, best_pruner_name = min(best_scores, key=lambda k: best_scores[k]['test_mse'])
        best_model_info = best_scores[(best_model_name, best_pruner_name)]
        print(f"\nBest model on test data: {best_model_name} with {best_pruner_name}")
        print(f"Test MSE: {best_model_info['test_mse']}")
        print(f"Test RMSE: {best_model_info['test_rmse']}")
        print(f"Correlation Coefficient: {best_model_info['test_corr_coef']}")
        print(f"Best Parameters: {best_model_info['best_params']}")
        print(f"Pruner Used: {best_model_info['pruner']}")
    else:
        print("No valid model configurations found.")

    return best_scores

# Example usage
best_scores_autosampler = hyperparameter_tuning_all(X_train, y_train, X_test, y_test, "./drive/MyDrive/streamflow/TDS/tds_ml_tabnet/HyperParameter_Tuning/test.xlsx")

Running Optuna for Random Forest with MedianPruner...
Best MSE for Random Forest with MedianPruner: 11899.272160644423 with params: {'n_estimators': 200, 'criterion': 'poisson', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 0.01, 'min_weight_fraction_leaf': 0.01, 'max_features': 'sqrt', 'max_leaf_nodes': 200, 'min_impurity_decrease': 0.2, 'n_jobs': -1, 'random_state': 42, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.001}
Best RMSE for Random Forest with MedianPruner: 109.08378504912828
Correlation Coefficient for Random Forest with MedianPruner: 0.8477575383559034
Running Optuna for Random Forest with NopPruner...
Best MSE for Random Forest with NopPruner: 12448.972212788854 with params: {'n_estimators': 300, 'criterion': 'poisson', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 0.01, 'min_weight_fraction_leaf': 0.01, 'max_features': 'sqrt', 'max_leaf_nodes': 50, 'min_impurity_decrease': 0.2, 'n_jobs': -1, 'random_state': 42, 'verbose': 0, 'warm_star

In [10]:
best_scores_autosampler

{('Random Forest', 'MedianPruner'): {'best_score': 11899.272160644423,
  'best_params': {'n_estimators': 200,
   'criterion': 'poisson',
   'max_depth': None,
   'min_samples_split': 2,
   'min_samples_leaf': 0.01,
   'min_weight_fraction_leaf': 0.01,
   'max_features': 'sqrt',
   'max_leaf_nodes': 200,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.001},
  'test_mse': 11899.272160644423,
  'test_rmse': 109.08378504912828,
  'test_corr_coef': 0.8477575383559034,
  'pruner': 'MedianPruner'},
 ('Random Forest', 'NopPruner'): {'best_score': 12448.972212788854,
  'best_params': {'n_estimators': 300,
   'criterion': 'poisson',
   'max_depth': 10,
   'min_samples_split': 5,
   'min_samples_leaf': 0.01,
   'min_weight_fraction_leaf': 0.01,
   'max_features': 'sqrt',
   'max_leaf_nodes': 50,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'cc

# **Best Model Analysis**

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pytorch_tabnet import TabNetRegressor  # As you confirmed, TabNet is available!

def get_best_models_and_predict(best_scores_autosampler, X_train, y_train, X_test, y_test, file_path):
    # Convert input data to NumPy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    # Model mapping, now including TabNet!
    model_mapping = {
        'Random Forest': RandomForestRegressor,
        'Gradient Boosting': GradientBoostingRegressor,
        'XGBoost': XGBRegressor,
        'LightGBM': LGBMRegressor,
        'CatBoost': CatBoostRegressor,
        'GPBoost': GPBoostRegressor,
        'NGBoost': NGBRegressor,
        'HistGradientBoosting': HistGradientBoostingRegressor,
        'PGBM': PGBM,
        'TabNet': TabNetRegressor
    }

    # Dictionary to store the best model for each type
    best_models = {}

    # Select the best pruner for each model type
    for (model_name, pruner), params in best_scores_autosampler.items():
        current_score = params.get('test_mse', np.inf)
        if model_name not in best_models or current_score < best_models[model_name]['score']:
            best_models[model_name] = {
                'score': current_score,
                'params': params['best_params'],
                'pruner': pruner
            }

    # Prepare DataFrame for predictions
    df = pd.read_csv(file_path)
    successful_plots = []

    # Iterate over the best models to train and predict
    for model_name, model_info in best_models.items():
        best_params = dict(model_info['params'])  # Shallow copy to avoid mutation
        model_class = model_mapping.get(model_name)
        if not model_class:
            print(f"Model {model_name} is not supported or not available. Skipping.")
            continue

        try:
            if model_name == 'PGBM':
                model = model_class()
                model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=best_params)
                predictions = model.predict(X_test)
            elif model_name == 'TabNet':
                # Handle TabNet data shape and training
                y_train_fit = y_train.reshape(-1, 1)
                model = model_class(**{k: v for k, v in best_params.items() if k != "verbose"})
                model.fit(X_train, y_train_fit, max_epochs=100, patience=10, batch_size=1024, eval_set=[(X_train, y_train_fit)])
                predictions = model.predict(X_test).flatten()
            else:
                if model_name == 'CatBoost':
                    best_params.pop('verbose', None)
                model = model_class(**best_params)
                model.fit(X_train, y_train)
                predictions = model.predict(X_test)
        except Exception as e:
            print(f"Failed to train or predict with model {model_name}: {e}")
            continue

        # Ensure predictions are 1D for plotting and DataFrame
        predictions = np.array(predictions).flatten()
        df[f'{model_name} Predictions'] = predictions

        # Plot actual vs. predicted
        try:
            plt.figure(figsize=(10, 6))
            plt.scatter(y_test, predictions, alpha=0.6)
            plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', color='red', lw=2)
            plt.xlabel("Actual")
            plt.ylabel("Predicted")
            plt.title(f"Actual vs. Predicted Values ({model_name})")
            plt.grid(True)
            plt.tight_layout()
            plot_path = f'temp_plot_{model_name}.png'
            plt.savefig(plot_path)
            plt.close()
            successful_plots.append((model_name, plot_path))
        except Exception as e:
            print(f"Plotting failed for {model_name}: {e}")

    # Save predictions and plots to Excel
    with pd.ExcelWriter(file_path.replace('.csv', '_results.xlsx'), engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Data', index=False)
        workbook = writer.book
        # Insert ONLY the images that actually exist
        for model_name, plot_path in successful_plots:
            if os.path.exists(plot_path):
                short_model_name = ''.join(word[0] for word in model_name.split())
                sheet_name = f'{short_model_name}_Plot'
                worksheet = workbook.add_worksheet(sheet_name)
                writer.sheets[sheet_name] = worksheet
                worksheet.insert_image('A1', plot_path)
            else:
                print(f"Plot image does not exist and will not be inserted for {model_name}: {plot_path}")

    # Clean up ONLY the files that were actually created
    for _, plot_path in successful_plots:
        if os.path.exists(plot_path):
            os.remove(plot_path)

    return df, best_models

# Call the function
df, best_models = get_best_models_and_predict(best_scores_autosampler, X_train, y_train, X_test, y_test, "./drive/MyDrive/streamflow/TDS/test.csv")

0:	learn: 279.2325090	total: 5.04ms	remaining: 1s
1:	learn: 274.6177976	total: 5.6ms	remaining: 555ms
2:	learn: 269.0627888	total: 5.8ms	remaining: 381ms
3:	learn: 265.0639962	total: 5.95ms	remaining: 292ms
4:	learn: 260.0592385	total: 6.09ms	remaining: 238ms
5:	learn: 255.1984039	total: 6.24ms	remaining: 202ms
6:	learn: 250.4520269	total: 6.42ms	remaining: 177ms
7:	learn: 245.5294657	total: 6.56ms	remaining: 157ms
8:	learn: 241.2354853	total: 6.71ms	remaining: 142ms
9:	learn: 237.2383014	total: 6.84ms	remaining: 130ms
10:	learn: 233.5596286	total: 6.98ms	remaining: 120ms
11:	learn: 229.7039990	total: 7.11ms	remaining: 111ms
12:	learn: 225.7110338	total: 7.26ms	remaining: 104ms
13:	learn: 222.2194395	total: 7.39ms	remaining: 98.2ms
14:	learn: 218.2656562	total: 7.52ms	remaining: 92.7ms
15:	learn: 214.2830025	total: 7.69ms	remaining: 88.4ms
16:	learn: 210.7511383	total: 7.83ms	remaining: 84.3ms
17:	learn: 207.4566555	total: 7.96ms	remaining: 80.5ms
18:	learn: 204.3293467	total: 8.09ms	r

In [12]:
plot_best_scores(best_scores_autosampler,"./drive/MyDrive/streamflow/TDS/tds_ml_tabnet/HyperParameter_Tuning/test_results.xlsx")

In [13]:
generate_interpretml_explanations_summary_pruners(best_scores_autosampler, X_train, y_train, x_test, feature_names,excel_file_path = "./drive/MyDrive/streamflow/TDS/tds_ml_tabnet/HyperParameter_Tuning/test_results.xlsx")

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

Model HistGradientBoosting is not supported or not available.
Model PGBM is not supported or not available.
epoch 0  | loss: 152222.96875| val_0_mse: 153868.59375|  0:00:00s
epoch 1  | loss: 150483.84375| val_0_mse: 149211.76562|  0:00:00s
epoch 2  | loss: 148098.54688| val_0_mse: 146123.4375|  0:00:00s
epoch 3  | loss: 146813.6875| val_0_mse: 142923.75|  0:00:00s
epoch 4  | loss: 145641.4375| val_0_mse: 139571.01562|  0:00:01s
epoch 5  | loss: 144214.85938| val_0_mse: 136519.20312|  0:00:01s
epoch 6  | loss: 142774.70312| val_0_mse: 130839.44531|  0:00:01s
epoch 7  | loss: 141427.65625| val_0_mse: 128234.24219|  0:00:01s
epoch 8  | loss: 140470.4375| val_0_mse: 125901.22656|  0:00:01s
epoch 9  | loss: 139392.51562| val_0_mse: 127233.92969|  0:00:01s
epoch 10 | loss: 137775.375| val_0_mse: 125370.09375|  0:00:01s
epoch 11 | loss: 136867.04688| val_0_mse: 123779.72656|  0:00:01s
epoch 12 | loss: 135679.01562| val_0_mse: 117037.60156|  0:00:01s
epoch 13 | loss: 134373.6875| val_0_mse: 11

  0%|          | 0/96 [00:00<?, ?it/s]